In [1]:
from PyQt5.QtWidgets import *
from PyQt5.QtCore import QCoreApplication
import sys
import platform
if platform.system() == "Windows":
    import winsound
import cv2
import numpy as np

## 6.2 PyQt를 이용한 사용자 인터페이스

In [2]:
class BeepSound(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("삑 소리 내기")
        self.setGeometry(200, 200, 500, 100)
        
        short_beep_button = QPushButton("짧게 삑", self)
        long_beep_button = QPushButton("길게 삑", self)
        quit_button = QPushButton("나가기", self)
        
        self.label = QLabel("환영합니다!", self)
        
        short_beep_button.setGeometry(10, 10, 100, 30)
        long_beep_button.setGeometry(110, 10, 100, 30)
        quit_button.setGeometry(210, 10, 100, 30)
        self.label.setGeometry(10, 40, 500, 70)
        
        short_beep_button.clicked.connect(self.short_beep_function)
        long_beep_button.clicked.connect(self.long_beep_function)
        quit_button.clicked.connect(self.quit_function)
        
    def short_beep_function(self):
        self.label.setText("주파수 1000으로 0.5초 동안 삑 소리를 냅니다.")
        winsound.Beep(1000, 500)
    
    def long_beep_function(self):
        self.label.setText("주파수 1000으로 3초 동안 삑 소리를 냅니다.")
        winsound.Beep(1000, 3000)
        
    def quit_function(self):
        self.close()
        

app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = BeepSound()
win.show()
app.exec_()

In [3]:
class Video(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("비디오에서 프레임 수집")
        self.setGeometry(200, 200, 500, 100)
        
        video_button = QPushButton("비디오 켜기", self)
        capture_button = QPushButton("프레임 잡기", self)
        save_button = QPushButton("프레임 저장", self)
        quit_button = QPushButton("나가기", self)
        
        video_button.setGeometry(10, 10, 100, 30)
        capture_button.setGeometry(110, 10, 100, 30)
        save_button.setGeometry(210, 10, 100, 30)
        quit_button.setGeometry(310, 10, 100, 30)
        
        video_button.clicked.connect(self.video_function)
        capture_button.clicked.connect(self.capture_function)
        save_button.clicked.connect(self.save_function)
        quit_button.clicked.connect(self.quit_function)
        
    def video_function(self):
        self.cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        if not self.cap.isOpened(): self.close()
        
        while True:
            ret, self.frame = self.cap.read()
            if not ret: break
            cv2.imshow("Video display", self.frame)
            cv2.waitKey(1)
        
    def capture_function(self):
        self.capture_frame = self.frame
        cv2.imshow("Captured Frame", self.frame)
    
    def save_function(self):
        f_name = QFileDialog.getSaveFileName(self, "파일 저장", "./")
        cv2.imwrite(f_name[0], self.capture_frame)
        
    def quit_function(self):
        self.cap.release()
        cv2.destroyAllWindows()
        self.close()
        

app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = Video()
win.show()
app.exec_()

0

## 6.3 오림

In [3]:
class Orim(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("오림")
        self.setGeometry(200, 200, 700, 200)
        
        file_button = QPushButton("파일", self)
        paint_button = QPushButton("페인팅", self)
        cut_button = QPushButton("오림", self)
        inc_button = QPushButton('+', self)
        dec_button = QPushButton('-', self)
        save_button = QPushButton("저장", self)
        quit_button = QPushButton("나가기", self)
        
        
        file_button.setGeometry(10, 10, 100, 30)
        paint_button.setGeometry(110, 10, 100, 30)
        cut_button.setGeometry(210, 10, 100, 30)
        inc_button.setGeometry(310, 10, 50, 30)
        dec_button.setGeometry(360, 10, 50, 30)
        save_button.setGeometry(410, 10, 100, 30)
        quit_button.setGeometry(510, 10, 100, 30)
        
        file_button.clicked.connect(self.file_open_function)
        paint_button.clicked.connect(self.painting_function)
        cut_button.clicked.connect(self.cut_function)
        inc_button.clicked.connect(self.inc_function)
        dec_button.clicked.connect(self.dec_function)
        save_button.clicked.connect(self.save_function)
        quit_button.clicked.connect(self.quit_function)
        
        self.brush_size = 5
        self.left_color, self.right_color = (255, 0, 0), (0, 0, 255)
        
    def file_open_function(self):
        f_name = QFileDialog.getOpenFileName(self, "open file", "./")
        self.img = cv2.imread(f_name[0])
        if self.img is None: sys.exit("파일을 찾을 수 없습니다.")
        
        self.img_show = np.copy(self.img)
        cv2.imshow("Painting", self.img_show)
        
        self.mask = np.zeros((self.img.shape[0], self.img.shape[1]), np.uint8)
        self.mask[:, :] = cv2.GC_PR_BGD
        
    def painting_function(self):
        cv2.setMouseCallback("Painting", self.painting)
        
    def painting(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.left_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        elif event == cv2.EVENT_RBUTTONDOWN:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.right_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.left_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_RBUTTON:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.right_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        
        cv2.imshow("Painting", self.img_show)
        
    def cut_function(self):
        background = np.zeros((1, 65), np.float64)
        foreground = np.zeros((1, 65), np.float64)
        
        cv2.grabCut(self.img, self.mask, None, background, foreground, 5, cv2.GC_INIT_WITH_MASK)
        mask2 = np.where((self.mask == 2) | (self.mask == 0), 0, 1).astype("uint8")
        self.grabImg = self.img*mask2[..., np.newaxis]
        cv2.imshow("Scissoring", self.grabImg)
        
    def inc_function(self):
        self.brush_size = min(20, self.brush_size + 1)
        
    def dec_function(self):
        self.brush_size = max(1, self.brush_size - 1)
        
    def save_function(self):
        f_name = QFileDialog.getSaveFileName(self, "파일 저장", './')
        cv2.imwrite(f_name[0], self.grabImg)
        
    def quit_function(self):
        cv2.destroyAllWindows()
        self.close()

        
app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = Orim()
win.show()
app.exec_()

0

## 6.4 교통약자 보호구역 알림

In [8]:
class TrafficWeek(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("교통약자 보호")
        self.setGeometry(200, 200, 700, 200)
        
        sign_button = QPushButton("표지판 등록", self)
        road_button = QPushButton("도로 영상 불러옴", self)
        recognition_button = QPushButton("인식", self)
        quit_button = QPushButton("나가기", self)
        self.label = QLabel("환영합니다!", self)
        
        sign_button.setGeometry(10, 10, 100, 30)
        road_button.setGeometry(110, 10, 100, 30)
        recognition_button.setGeometry(210, 10, 100, 30)
        quit_button.setGeometry(310, 10, 100 ,30)
        self.label.setGeometry(10, 40, 600, 170)
        
        sign_button.clicked.connect(self.sign_function)
        road_button.clicked.connect(self.road_function)
        recognition_button.clicked.connect(self.recognition_function)
        quit_button.clicked.connect(self.quit_function)
        
        self.sign_files = [["./imgs/child.png", "어린이"], ["./imgs/elder.png", "노인"], ["./imgs/disabled.png", "장애인"], ["./imgs/model.png", "간판"]]
        self.sign_imgs = []
        
    def sign_function(self):
        self.label.clear()
        self.label.setText("교통약자 표지판을 등록합니다.")
        
        for fname, _ in self.sign_files:
            self.sign_imgs.append(cv2.imread(fname))
            cv2.imshow(fname, self.sign_imgs[-1])
        
    def road_function(self):
        if self.sign_imgs == []:
            self.label.setText("먼저 표지판을 등록하세요.")
        else:
            fname = QFileDialog.getOpenFileName(self, "파일 읽기", "./")
            self.road_img = cv2.imread(fname[0])
            if self.road_img is None: sys.exit("파일을 찾을 수 없습니다.")
        
        cv2.imshow("Road scen", self.road_img)
    
    def recognition_function(self):
        if self.road_img is None:
            self.label.setText("먼저 도로 영상을 입력하세요")
        else:
            sift = cv2.SIFT_create()
        
        KD = []
        for img in self.sign_imgs:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            KD.append(sift.detectAndCompute(gray, None))
        
        gray_road = cv2.cvtColor(self.road_img, cv2.COLOR_BGR2GRAY)
        road_kp, road_des = sift.detectAndCompute(gray_road, None)
        
        matcher = cv2.DescriptorMatcher_create(cv2.DescriptorMatcher_FLANNBASED)
        GM = []
        for sign_kp, sign_des in KD:
            knn_match = matcher.knnMatch(sign_des, road_des, 2)
            T = 0.7
            good_match = []
            for nearest1, nearest2 in knn_match:
                if (nearest1.distance / nearest2.distance) < T:
                    good_match.append(nearest1)
            GM.append(good_match)
        
        best = GM.index(max(GM, key=len))
        
        if len(GM[best]) < 4:
            self.label.setText("표지판이 없습니다.")
        else:
            sign_kp = KD[best][0]
            good_match = GM[best]
            
            points1 = np.float32([sign_kp[gm.queryIdx].pt for gm in good_match])
            points2 = np.float32([road_kp[gm.trainIdx].pt for gm in good_match])
            
            H, _ = cv2.findHomography(points1, points2, cv2.RANSAC)
            
            h1, w1 = self.sign_imgs[best].shape[0], self.sign_imgs[best].shape[1]
            h2, w2 = self.road_img.shape[0], self.road_img.shape[1]
            
            box1 = np.float32([[0, 0], [0, h1-1], [w1-1, h1-1], [w1-1, 0]]).reshape(4, 1, 2)
            box2 = cv2.perspectiveTransform(box1, H)
            
            self.road_img = cv2.polylines(self.road_img, [np.int32(box2)], True, (0, 255, 0), 4)
            
            img_match = np.empty((max(h1, h2), w1+w2, 3), dtype=np.uint8)
            
            cv2.drawMatches(self.sign_imgs[best], sign_kp, self.road_img, road_kp, good_match, img_match,
                           flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
            
            cv2.imshow("Matches and Homgraphy", img_match)
            
            self.label.setText(self.sign_files[best][1] + "보호구역입니다. 30km로 서행하세요")
            winsound.Beep(3000, 500)
        
    def quit_function(self):
        cv2.destroyAllWindows()
        self.close()

            

            
app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = TrafficWeek()
win.show()
app.exec_()            

0

## 6.5 파노라마 영상 제작

In [14]:
class Panorama(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("파노마라 영상")
        self.setGeometry(200, 200, 700, 200)
        self.cap = None
        
        collect_button = QPushButton("영상 수집", self)
        self.show_button = QPushButton("영상 보기", self)
        self.stitch_button = QPushButton("봉합", self)
        self.save_button = QPushButton("저장", self)
        quit_button = QPushButton("나가기", self)
        self.label = QLabel("환영합니다.", self)
        
        collect_button.setGeometry(10, 25, 100, 30)
        self.show_button.setGeometry(110, 25, 100, 30)
        self.stitch_button.setGeometry(210, 25, 100, 30)
        self.save_button.setGeometry(310, 25, 100, 30)
        quit_button.setGeometry(450, 25, 100, 30)
        self.label.setGeometry(10, 70, 600, 170)
        
        self.show_button.setEnabled(False)
        self.stitch_button.setEnabled(False)
        self.save_button.setEnabled(False)
        
        collect_button.clicked.connect(self.colloect_function)
        self.show_button.clicked.connect(self.show_fucntion)
        self.stitch_button.clicked.connect(self.stitch_function)
        self.save_button.clicked.connect(self.save_function)
        quit_button.clicked.connect(self.quit_function)
        
    def colloect_function(self):
        self.show_button.setEnabled(False)
        self.stitch_button.setEnabled(False)
        self.save_button.setEnabled(False)
        self.label.setText("c를 여러 번 눌러 수집하고 끝나면 q를 눌러 비디오를 끕니다.")
        
        self.cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        if not self.cap.isOpened(): sys.eixt("카메라 연결 실패")
        
        self.imgs = []
        while True:
            ret, frame = self.cap.read()
            if not ret: break
            
            cv2.imshow("video display", frame)
            
            key = cv2.waitKey(1)
            if key == ord('c'):
                self.imgs.append(frame)
            elif key == ord('q'):
                self.cap.release()
                cv2.destroyWindow("video display")
                break
        
        if len(self.imgs) >= 2:
            self.show_button.setEnabled(True)
            self.stitch_button.setEnabled(True)
            self.save_button.setEnabled(True)
            
    def show_fucntion(self):
        self.label.setText("수집된 영상은 "+str(len(self.imgs))+"장 입니다.")
        stack = cv2.resize(self.imgs[0], dsize=(0, 0), fx=0.25, fy=0.25)
        for i in range(1, len(self.imgs)):
            stack = np.hstack((stack, cv2.resize(self.imgs[i], dsize=(0, 0), fx=0.25, fy=0.25)))
        cv2.imshow("Iamge collection", stack)
        
    def stitch_function(self):
        stitcher = cv2.Stitcher_create()
        status, self.img_stitched = stitcher.stitch(self.imgs)
        if status == cv2.STITCHER_OK:
            cv2.imshow("Image stitched panorama", self.img_stitched)
        else:
            winsound.Beep(3000, 500)
            self.label.setText("파노라마 제작에 실패했습니다. 다시 시도하세요.")
            
    def save_function(self):
        fname = QFileDialog.getSaveFileName(self, "파일 저장", "./")
        cv2.imwirte(fname[0], self.img_stitched)
        
    def quit_function(self):
        if self.cap:
            self.cap.release()
        cv2.destroyAllWindows()
        self.close()
        

app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = Panorama()
win.show()
app.exec_()             

0

## 6.6 특수 효과

In [7]:
class SpecialEffect(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("사진 특수 효과")
        self.setGeometry(200, 200, 800, 200)
        picture_button = QPushButton("사진 읽기", self)
        emboss_button = QPushButton("엠보싱", self)
        cartoon_button = QPushButton("카툰", self)
        sketch_button = QPushButton("스케치", self)
        oil_button = QPushButton("유화", self)
        save_button = QPushButton("저장", self)
        self.pick_combo = QComboBox(self)
        self.pick_combo.addItems(["엠보싱", "카툰", "스케치(명암)", "스케치(컬러)", "유화"])
        quit_button = QPushButton("나가기", self)
        self.label = QLabel("환영합니다.", self)
        
        picture_button.setGeometry(10, 10, 100, 30)
        emboss_button.setGeometry(110, 10, 100 ,30)
        cartoon_button.setGeometry(210, 10, 100, 30)
        sketch_button.setGeometry(310, 10, 100, 30)
        oil_button.setGeometry(410, 10, 100, 30)
        save_button.setGeometry(510, 10, 100, 30)
        self.pick_combo.setGeometry(510, 40, 110, 30)
        quit_button.setGeometry(620, 10, 100, 30)
        self.label.setGeometry(10, 40, 500, 170)
        
        picture_button.clicked.connect(self.picture_open_fucntion)
        emboss_button.clicked.connect(self.emboss_function)
        cartoon_button.clicked.connect(self.cartoon_function)
        sketch_button.clicked.connect(self.sketch_function)
        oil_button.clicked.connect(self.oil_function)
        save_button.clicked.connect(self.save_function)
        quit_button.clicked.connect(self.quit_fucntion)
        
    def picture_open_fucntion(self):
        fname = QFileDialog.getOpenFileName(self, "사진 읽기", "./")
        self.img = cv2.imread(fname[0])
        if self.img is None: sys.eixt("파일을 찾을 수 없습니다.")
        
        cv2.imshow("Painting", self.img)
        
    def emboss_function(self):
        femboss = np.array([[-1.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 1.0]])
        
        gray = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)
        gray16 = np.int16(gray)
        self.emboss = np.uint8(np.clip(cv2.filter2D(gray16, -1, femboss)+128, 0, 255))
        
        cv2.imshow("Emboss", self.emboss)
        
    def cartoon_function(self):
        self.cartoon = cv2.stylization(self.img, sigma_s=60, sigma_r=0.45)
        cv2.imshow("Cartoon", self.cartoon)
        
    def sketch_function(self):
        self.sketch_gray, self.sketch_color = cv2.pencilSketch(self.img, sigma_s=60, sigma_r=0.07, shade_factor=0.02)
        cv2.imshow("sketch(gray)", self.sketch_gray)
        cv2.imshow("sketch(color)", self.sketch_color)
        
    def oil_function(self):
        self.oil = cv2.xphoto.oilPainting(self.img, 10, 1, cv2.COLOR_BGR2Lab)
        cv2.imshow("Oil", self.oil)
        
    def save_function(self):
        fname = QFileDialog.getSaveFileName(self, "파일 저장", "./")
    
        i = self.pick_combo.currentIndex()
        if i == 0: cv2.imwrite(fname[0], self.emboss)
        elif i == 1: cv2.imwrite(fname[0], self.cartoon)
        elif i == 2: cv2.imwrite(fname[0], self.sketch_gray)
        elif i == 3: cv2.imwrite(fname[0], self.sketch_color)
        elif i == 4: cv2.imwrite(fname[0], self.oil)
        
    def quit_fucntion(self):
        cv2.destroyAllWindows()
        self.close()
        

app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = SpecialEffect()
win.show()
app.exec_()    

0

In [2]:
class VideoSpecialEffect(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("비디오 특수 효과")
        self.setGeometry(200, 200, 400, 100)
        
        video_button = QPushButton("비디오 시작", self)
        self.pick_combo = QComboBox(self)
        self.pick_combo.addItems(["엠보싱", "카툰", "스케치(명암)", "스케치(컬러)", "유화"])
        quit_button = QPushButton("나가기", self)
        
        video_button.setGeometry(10, 10, 140, 30)
        self.pick_combo.setGeometry(150, 10, 110, 30)
        quit_button.setGeometry(280, 10, 100, 30)
        
        video_button.clicked.connect(self.video_special_effect_function)
        quit_button.clicked.connect(self.quit_function)
        
    def video_special_effect_function(self):
        self.cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        if not self.cap.isOpened(): sys.exit("카메라 연결 실패")
        
        while True:
            ret, frame = self.cap.read()
            if not ret: break
            
            pick_effect = self.pick_combo.currentIndex()
            if pick_effect == 0:
                femboss = np.array([[-1.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 1.0]])
                gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                gray16 = np.int16(gray)
                special_img = np.uint8(np.clip(cv2.filter(gray, 16, -1, femboss) + 128, 0, 255))
            elif pick_effect == 1:
                special_img = cv2.stylization(frame, sigma_s=60, sigma_r=0.45)
            elif pick_effect == 2:
                special_img, _ = cv2.pencilSketch(frame, sigma_s=60, sigma_r=0.07, shade_factor=0.02)
            elif pick_effect ==3:
                _, special_img = cv2.pencilSketch(frame, sigma_s=60, sigma_r=0.07, shade_factor=0.02)
            elif pick_effect == 4:
                special_img = cv2.xphoto.oilPainting(frame, 10, 1, cv2.COLOR_BGR2Lab)
        
        cv2.imshow("Special effect", special_img)
        cv2.waitKey(1)
    
    def quit_function(self):
        self.cap.release()
        cv2.destroyAllWindows()
        self.close()


        
app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = VideoSpecialEffect()
win.show()
app.exec_()  

SyntaxError: invalid syntax (1881431413.py, line 16)

## 연습문제

In [ ]:
# 문제 1
class BeepSound(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("삑 소리 내기")
        self.setGeometry(200, 200, 500, 100)
        
        short_beep_button = QPushButton("짧게 삑", self)
        long_beep_button = QPushButton("길게 삑", self)
        play_sound_button = QPushButton("Play Sound", self)
        quit_button = QPushButton("나가기", self)
        
        self.label = QLabel("환영합니다!", self)
        
        short_beep_button.setGeometry(10, 10, 100, 30)
        long_beep_button.setGeometry(110, 10, 100, 30)
        play_sound_button.setGeometry(210, 10, 100, 30)
        quit_button.setGeometry(310, 10, 100, 30)
        self.label.setGeometry(10, 40, 500, 70)
        
        short_beep_button.clicked.connect(self.short_beep_function)
        long_beep_button.clicked.connect(self.long_beep_function)
        play_sound_button.clicked.connect(self.play_sound_function)
        quit_button.clicked.connect(self.quit_function)
        
    def short_beep_function(self):
        self.label.setText("주파수 1000으로 0.5초 동안 삑 소리를 냅니다.")
        winsound.Beep(1000, 500)
    
    def long_beep_function(self):
        self.label.setText("주파수 1000으로 3초 동안 삑 소리를 냅니다.")
        winsound.Beep(1000, 3000)
        
    def play_sound_function(self):
        self.label.setText("play sound 소리를 냅니다.")
        winsound.PlaySound()
        
    def quit_function(self):
        self.close()
        

app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = BeepSound()
win.show()
app.exec_()

In [ ]:
# 문제 2
class Video(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("비디오에서 프레임 수집")
        self.setGeometry(200, 200, 500, 100)
        
        video_button = QPushButton("비디오 켜기", self)
        capture_button = QPushButton("프레임 잡기", self)
        multi_capture_button = QPushButton("여러 프레임 잡기", self)
        save_button = QPushButton("프레임 저장", self)
        quit_button = QPushButton("나가기", self)
        
        video_button.setGeometry(10, 10, 100, 30)
        capture_button.setGeometry(110, 10, 100, 30)
        multi_capture_button.setGeometry(210, 10, 100, 30)
        save_button.setGeometry(310, 10, 100, 30)
        quit_button.setGeometry(410, 10, 100, 30)
        
        video_button.clicked.connect(self.video_function)
        capture_button.clicked.connect(self.capture_function)
        multi_capture_button.clicked.connect(self.multi_capture_function)
        save_button.clicked.connect(self.save_function)
        quit_button.clicked.connect(self.quit_function)
        
    def video_function(self):
        self.cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        if not self.cap.isOpened(): self.close()
        
        while True:
            ret, self.frame = self.cap.read()
            if not ret: break
            cv2.imshow("Video display", self.frame)
            self.key = cv2.waitKey(1)
        
    def capture_function(self):
        self.capture_frame = self.frame
        cv2.imshow("Captured Frame", self.frame)
        
    def multi_capture_function(self):
        frames = []
        while True:
            if self.key == ord('c'):
                frames.append(self.frame)
            elif self.key == ord('q'):
                break
        cv2.imshow("Multi Captured Frame", np.hstack(frames))
    
    def save_function(self):
        f_name = QFileDialog.getSaveFileName(self, "파일 저장", "./")
        cv2.imwrite(f_name[0], self.capture_frame)
        
    def quit_function(self):
        self.cap.release()
        cv2.destroyAllWindows()
        self.close()
        

app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = Video()
win.show()
app.exec_()

In [ ]:
# 문제 3
# (1)
'''
self.mask가 파일을 불러올 때 선언되기 때문.
'''
## (2)
class Orim(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("오림")
        self.setGeometry(200, 200, 700, 200)
        
        file_button = QPushButton("파일", self)
        paint_button = QPushButton("페인팅", self)
        cut_button = QPushButton("오림", self)
        inc_button = QPushButton('+', self)
        dec_button = QPushButton('-', self)
        save_button = QPushButton("저장", self)
        quit_button = QPushButton("나가기", self)
        
        file_button.setGeometry(10, 10, 100, 30)
        paint_button.setGeometry(110, 10, 100, 30)
        cut_button.setGeometry(210, 10, 100, 30)
        inc_button.setGeometry(310, 10, 50, 30)
        dec_button.setGeometry(360, 10, 50, 30)
        save_button.setGeometry(410, 10, 100, 30)
        quit_button.setGeometry(510, 10, 100, 30)
        
        file_button.clicked.connect(self.file_open_function)
        paint_button.clicked.connect(self.painting_function)
        cut_button.clicked.connect(self.cut_function)
        inc_button.clicked.connect(self.inc_function)
        dec_button.clicked.connect(self.dec_function)
        save_button.clicked.connect(self.save_function)
        quit_button.clicked.connect(self.quit_function)
        
        self.brush_size = 5
        self.left_color, self.right_color = (255, 0, 0), (0, 0, 255)
        self.mask = None
        
    def file_open_function(self):
        f_name = QFileDialog.getOpenFileName(self, "open file", "./")
        self.img = cv2.imread(f_name[0])
        if self.img is None: sys.exit("파일을 찾을 수 없습니다.")
        
        self.img_show = np.copy(self.img)
        cv2.imshow("Painting", self.img_show)
        
        self.mask = np.zeros((self.img.shape[0], self.img.shape[1]), np.uint8)
        self.mask[:, :] = cv2.GC_PR_BGD
        
    def painting_function(self):
        cv2.setMouseCallback("Painting", self.painting)
        
    def painting(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.left_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        elif event == cv2.EVENT_RBUTTONDOWN:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.right_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.left_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_RBUTTON:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.right_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        
        cv2.imshow("Painting", self.img_show)
        
    def cut_function(self):
        if self.mask:
            background = np.zeros((1, 65), np.float64)
            foreground = np.zeros((1, 65), np.float64)

            cv2.grabCut(self.img, self.mask, None, background, foreground, 5, cv2.GC_INIT_WITH_MASK)
            mask2 = np.where((self.mask == 2) | (self.mask == 0), 0, 1).astype("uint8")
            self.grabImg = self.img*mask2[..., np.newaxis]
            cv2.imshow("Scissoring", self.grabImg)
        
    def inc_function(self):
        self.brush_size = min(20, self.brush_size + 1)
        
    def dec_function(self):
        self.brush_size = max(1, self.brush_size - 1)
        
    def save_function(self):
        f_name = QFileDialog.getSaveFileName(self, "파일 저장", './')
        cv2.imwrite(f_name[0], self.grabImg)
        
    def quit_function(self):
        cv2.destroyAllWindows()
        self.close()

        
app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = Orim()
win.show()
app.exec_()

In [ ]:
# 문제 4
class Orim(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("오림")
        self.setGeometry(200, 200, 700, 200)
        
        file_button = QPushButton("파일", self)
        paint_button = QPushButton("페인팅", self)
        clear_button = QPushButton("초기화", self)
        cut_button = QPushButton("오림", self)
        inc_button = QPushButton('+', self)
        dec_button = QPushButton('-', self)
        save_button = QPushButton("저장", self)
        quit_button = QPushButton("나가기", self)
        
        file_button.setGeometry(10, 10, 100, 30)
        paint_button.setGeometry(110, 10, 100, 30)
        cut_button.setGeometry(210, 10, 100, 30)
        inc_button.setGeometry(310, 10, 50, 30)
        dec_button.setGeometry(360, 10, 50, 30)
        save_button.setGeometry(410, 10, 100, 30)
        quit_button.setGeometry(510, 10, 100, 30)
        
        file_button.clicked.connect(self.file_open_function)
        paint_button.clicked.connect(self.painting_function)
        clear_button.clicked.connect(self.clear_function)
        cut_button.clicked.connect(self.cut_function)
        inc_button.clicked.connect(self.inc_function)
        dec_button.clicked.connect(self.dec_function)
        save_button.clicked.connect(self.save_function)
        quit_button.clicked.connect(self.quit_function)
        
        self.brush_size = 5
        self.left_color, self.right_color = (255, 0, 0), (0, 0, 255)
        self.mask = None
        
    def file_open_function(self):
        f_name = QFileDialog.getOpenFileName(self, "open file", "./")
        self.img = cv2.imread(f_name[0])
        if self.img is None: sys.exit("파일을 찾을 수 없습니다.")
        
        self.img_show = np.copy(self.img)
        cv2.imshow("Painting", self.img_show)
        
        self.mask = np.zeros((self.img.shape[0], self.img.shape[1]), np.uint8)
        self.mask[:, :] = cv2.GC_PR_BGD
        
    def painting_function(self):
        cv2.setMouseCallback("Painting", self.painting)
        
    def painting(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.left_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        elif event == cv2.EVENT_RBUTTONDOWN:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.right_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_LBUTTON:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.left_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        elif event == cv2.EVENT_MOUSEMOVE and flags == cv2.EVENT_FLAG_RBUTTON:
            cv2.circle(self.img_show, (x, y), self.brush_size, self.right_color, -1)
            cv2.circle(self.mask, (x, y), self.brush_size, cv2.GC_FGD, -1)
        
        cv2.imshow("Painting", self.img_show)
    
    def clear_function(self):
        self.img_show = self.img.copy()
        cv2.imshow("Painting", self.img_show)
        
    def cut_function(self):
        if self.mask:
            background = np.zeros((1, 65), np.float64)
            foreground = np.zeros((1, 65), np.float64)

            cv2.grabCut(self.img, self.mask, None, background, foreground, 5, cv2.GC_INIT_WITH_MASK)
            mask2 = np.where((self.mask == 2) | (self.mask == 0), 0, 1).astype("uint8")
            self.grabImg = self.img*mask2[..., np.newaxis]
            cv2.imshow("Scissoring", self.grabImg)
        
    def inc_function(self):
        self.brush_size = min(20, self.brush_size + 1)
        
    def dec_function(self):
        self.brush_size = max(1, self.brush_size - 1)
        
    def save_function(self):
        f_name = QFileDialog.getSaveFileName(self, "파일 저장", './')
        cv2.imwrite(f_name[0], self.grabImg)
        
    def quit_function(self):
        cv2.destroyAllWindows()
        self.close()

        
app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = Orim()
win.show()
app.exec_()

In [ ]:
# 문제 5
class Panorama(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("파노마라 영상")
        self.setGeometry(200, 200, 700, 200)
        self.cap = None
        
        collect_button = QPushButton("영상 수집", self)
        self.show_button = QPushButton("영상 보기", self)
        self.stitch_button = QPushButton("봉합", self)
        self.remove_button = QPushButton("가장자리 제거", self)
        self.save_button = QPushButton("저장", self)
        quit_button = QPushButton("나가기", self)
        self.label = QLabel("환영합니다.", self)
        
        collect_button.setGeometry(10, 25, 100, 30)
        self.show_button.setGeometry(110, 25, 100, 30)
        self.stitch_button.setGeometry(210, 25, 100, 30)
        self.remove_button.setGeometry(310, 25, 100, 30)
        self.save_button.setGeometry(410, 25, 100, 30)
        quit_button.setGeometry(550, 25, 100, 30)
        self.label.setGeometry(10, 70, 600, 170)
        
        self.show_button.setEnabled(False)
        self.stitch_button.setEnabled(False)
        self.remove_button.setEnabled(False)
        self.save_button.setEnabled(False)
        
        collect_button.clicked.connect(self.colloect_function)
        self.show_button.clicked.connect(self.show_fucntion)
        self.stitch_button.clicked.connect(self.stitch_function)
        self.remove_button.clicked.connect(self.remove_function)
        self.save_button.clicked.connect(self.save_function)
        quit_button.clicked.connect(self.quit_function)
        
    def colloect_function(self):
        self.show_button.setEnabled(False)
        self.stitch_button.setEnabled(False)
        self.save_button.setEnabled(False)
        self.label.setText("c를 여러 번 눌러 수집하고 끝나면 q를 눌러 비디오를 끕니다.")
        
        self.cap = cv2.VideoCapture(0, cv2.CAP_DSHOW)
        if not self.cap.isOpened(): sys.eixt("카메라 연결 실패")
        
        self.imgs = []
        while True:
            ret, frame = self.cap.read()
            if not ret: break
            
            cv2.imshow("video display", frame)
            
            key = cv2.waitKey(1)
            if key == ord('c'):
                self.imgs.append(frame)
            elif key == ord('q'):
                self.cap.release()
                cv2.destroyWindow("video display")
                break
        
        if len(self.imgs) >= 2:
            self.show_button.setEnabled(True)
            self.stitch_button.setEnabled(True)
            self.save_button.setEnabled(True)
            
    def show_fucntion(self):
        self.label.setText("수집된 영상은 "+str(len(self.imgs))+"장 입니다.")
        stack = cv2.resize(self.imgs[0], dsize=(0, 0), fx=0.25, fy=0.25)
        for i in range(1, len(self.imgs)):
            stack = np.hstack((stack, cv2.resize(self.imgs[i], dsize=(0, 0), fx=0.25, fy=0.25)))
        cv2.imshow("Iamge collection", stack)
        
    def stitch_function(self):
        self.remove_button.setEnabled(True)
        stitcher = cv2.Stitcher_create()
        status, self.img_stitched = stitcher.stitch(self.imgs)
        if status == cv2.STITCHER_OK:
            cv2.imshow("Image stitched panorama", self.img_stitched)
        else:
            winsound.Beep(3000, 500)
            self.label.setText("파노라마 제작에 실패했습니다. 다시 시도하세요.")
    
    def remove_function(self):
        thresh = cv2.threshold(cv2.cvtColor(self.img_stitched, cv2.COLOR_BGR2GRAY), 10, 255, cv2.THRESH_BINARY)[1]
        thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, np.ones((3, 3), np.uint8))
        
        cnts = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2] # Use index -2 for OpenCV 3 and 4 compatibility.
        c = cnts[0]  # Get the first contour (get the single contour).
        cv2.drawContours(thresh, [c], 0, 255, -1)  # Fill the contour with 255 (fill dark pixels under threshold inside the contour).

        # Find rectangle with minimum area
        rect = cv2.minAreaRect(c)
        box = np.int0(cv2.boxPoints(rect)) # Find 4 corners all convert from floating point values to int

        # Crop the rectangle with minimum area (crop both img and thresh)
        (topy, topx) = (np.min(box[:,1]), np.min(box[:,0]))  # https://stackoverflow.com/questions/28759253/how-to-crop-the-internal-area-of-a-contour
        (boty, botx) = (np.max(box[:,1]), np.max(box[:,0]))
        img = img[topy:boty+1, topx:botx+1, :]
        thresh = thresh[topy:boty+1, topx:botx+1]
        
        inv_thresh = 255 - thresh  # Inverse of thresh.

        cnts = cv2.findContours(inv_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)[-2]

        inv_thresh_bgr = cv2.cvtColor(inv_thresh, cv2.COLOR_GRAY2BGR)  # inv_thresh_bgr is used for testing.

        # Initialize top left and bottom right coordinates to be cropped (values are going apply area without black borders).
        x0 = 0
        y0 = 0
        x1 = img.shape[1]
        y1 = img.shape[0]

        # Iterate contours:
        for c in cnts:    
            (x, y, w, h) = cv2.boundingRect(c)  # Get contour bounding box.
            M = cv2.moments(c);cx = int(M["m10"] / M["m00"]);cy = int(M["m01"] / M["m00"]) # Compute the center of the contour (for testing)

            if (x == 0) and (h > w):
                # Enter here if "c" is the "Left contour" (for the given sample input, code should not reach here)
                x0 = w  # x0 is deffined by the left contour (x0 is the extreme right of the left contour).
            elif ((x + w) == img.shape[1]) and (h > w):
                # Enter here if "c" is the "Right contour"
                x1 = x  # x1 is deffined by the right contour (x1 is the extreme left coordinate of the right contour)
            elif (y == 0) and (w > h):
                # Enter here if "c" is the "Top contour"
                y0 = h  # y0 is deffined by the to contour (y0 is the extreme bottom of the top contour).
            elif ((y + h) == img.shape[0]) and (w > h):
                # Enter here if "c" is the "Bottom contour"
                y1 = y  # y1 is deffined by the bottom contour (y1 is the extreme top of the bottom contour).
        self.crop_img = self.img_stitched[y0:y1, x0:x1, :]
        cv2.imshow("Crop Image stitched panorama", self.crop_img)
        
    def save_function(self):
        fname = QFileDialog.getSaveFileName(self, "파일 저장", "./")
        cv2.imwirte(fname[0], self.img_stitched)
        
    def quit_function(self):
        if self.cap:
            self.cap.release()
        cv2.destroyAllWindows()
        self.close()
        

app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = Panorama()
win.show()
app.exec_()             

In [ ]:
# 문제 6
class SpecialEffect(QMainWindow):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("사진 특수 효과")
        self.setGeometry(200, 200, 800, 200)
        picture_button = QPushButton("사진 읽기", self)
        emboss_button = QPushButton("엠보싱", self)
        cartoon_button = QPushButton("카툰", self)
        sketch_button = QPushButton("스케치", self)
        oil_button = QPushButton("유화", self)
        motion_blur_button = QPushButton("모션 블러", self)
        save_button = QPushButton("저장", self)
        self.pick_combo = QComboBox(self)
        self.pick_combo.addItems(["엠보싱", "카툰", "스케치(명암)", "스케치(컬러)", "유화"])
        quit_button = QPushButton("나가기", self)
        self.label = QLabel("환영합니다.", self)
        
        picture_button.setGeometry(10, 10, 100, 30)
        emboss_button.setGeometry(110, 10, 100 ,30)
        cartoon_button.setGeometry(210, 10, 100, 30)
        sketch_button.setGeometry(310, 10, 100, 30)
        oil_button.setGeometry(410, 10, 100, 30)
        motion_blur_button.setGeometry(510, 10, 100, 30)
        save_button.setGeometry(510, 10, 100, 30)
        self.pick_combo.setGeometry(610, 40, 110, 30)
        quit_button.setGeometry(720, 10, 100, 30)
        self.label.setGeometry(10, 40, 500, 170)
        
        picture_button.clicked.connect(self.picture_open_fucntion)
        emboss_button.clicked.connect(self.emboss_function)
        cartoon_button.clicked.connect(self.cartoon_function)
        sketch_button.clicked.connect(self.sketch_function)
        oil_button.clicked.connect(self.oil_function)
        save_button.clicked.connect(self.save_function)
        quit_button.clicked.connect(self.quit_fucntion)
        
    def picture_open_fucntion(self):
        fname = QFileDialog.getOpenFileName(self, "사진 읽기", "./")
        self.img = cv2.imread(fname[0])
        if self.img is None: sys.eixt("파일을 찾을 수 없습니다.")
        
        cv2.imshow("Painting", self.img)
        
    def emboss_function(self):
        femboss = np.array([[-1.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 1.0]])
        
        gray = cv2.cvtColor(self.img, cv2.COLOR_BGR2GRAY)
        gray16 = np.int16(gray)
        self.emboss = np.uint8(np.clip(cv2.filter2D(gray16, -1, femboss)+128, 0, 255))
        
        cv2.imshow("Emboss", self.emboss)
        
    def cartoon_function(self):
        self.cartoon = cv2.stylization(self.img, sigma_s=60, sigma_r=0.45)
        cv2.imshow("Cartoon", self.cartoon)
        
    def sketch_function(self):
        self.sketch_gray, self.sketch_color = cv2.pencilSketch(self.img, sigma_s=60, sigma_r=0.07, shade_factor=0.02)
        cv2.imshow("sketch(gray)", self.sketch_gray)
        cv2.imshow("sketch(color)", self.sketch_color)
        
    def oil_function(self):
        self.oil = cv2.xphoto.oilPainting(self.img, 10, 1, cv2.COLOR_BGR2Lab)
        cv2.imshow("Oil", self.oil)
    
    def motion_blur_function(self):
        blur = np.zeros(3, 20) + (1 / 20)
        color16 = np.int16(self.img)
        self.blur = np.uint8(np.clip(cv2.filter2D(color16, -1, blur)+128, 0, 255))
        cv2.imshow("Blur", self.blur)
        
    def save_function(self):
        fname = QFileDialog.getSaveFileName(self, "파일 저장", "./")
    
        i = self.pick_combo.currentIndex()
        if i == 0: cv2.imwrite(fname[0], self.emboss)
        elif i == 1: cv2.imwrite(fname[0], self.cartoon)
        elif i == 2: cv2.imwrite(fname[0], self.sketch_gray)
        elif i == 3: cv2.imwrite(fname[0], self.sketch_color)
        elif i == 4: cv2.imwrite(fname[0], self.oil)
        
    def quit_fucntion(self):
        cv2.destroyAllWindows()
        self.close()
        

app = QCoreApplication.instance()
if app is None:
    app = QApplication(sys.argv)
win = SpecialEffect()
win.show()
app.exec_()    